
Two circles: alpha filtration persistent homology and persistence rings.

This example generates a noisy point cloud consisting of two disjoint circles in
$\mathbb R^2$, computes persistent homology over $\mathbb F_2$ using an
alpha filtration induced by a Delaunay triangulation, and visualizes the resulting
barcode via persistence rings.

# Data model

## Two circles point cloud

Let $n \in \mathbb N$, radius $r>0$, centers $c_\ell,c_r \in \mathbb R^2$,
and noise level $\sigma \ge 0$. The point cloud is

\begin{align}X = \{c_\ell + r(\cos \theta_i, \sin \theta_i) : i=1,\dots,n\}
       \cup
       \{c_r + r(\cos \varphi_i, \sin \varphi_i) : i=1,\dots,n\},\end{align}

where $\theta_i,\varphi_i$ are sampled uniformly from $[0,2\pi)$. If
$\sigma>0$, each coordinate is perturbed by independent Gaussian noise
$\mathcal N(0,\sigma^2)$.

# Homology pipeline

## Delaunay complex and alpha filtration

Compute a Delaunay triangulation of $X$ and take the downward closed simplicial
complex generated by its maximal simplices up to a chosen dimension.

For each simplex $\sigma$, define its squared alpha value

\begin{align}\alpha^2(\sigma)
   =
   \min_{c \in \mathbb R^2} \max_{x\in V(\sigma)} \|x-c\|^2,\end{align}

the squared radius of the smallest enclosing ball of its vertices.

The alpha filtration is the family of subcomplexes

\begin{align}K_t = \{\sigma : \alpha^2(\sigma) \le t\}.\end{align}

## Persistent homology

Fix the field $\mathbb F_2$. The function
:func:`homolipop.pipeline.persistent_homology_from_points` is assumed to compute

- an admissible filtration order compatible with $\alpha^2$
- persistent homology via standard boundary matrix reduction over $\mathbb F_2$
- a barcode object encoding the resulting persistence intervals

# Visualization

The function :func:`homolipop.persistence_rings.plot_rings_from_barcode` renders the
barcode using a concentric ring representation. Styling is controlled by
:class:`homolipop.persistence_rings.RingStyle`.

# Reproducibility

All randomness is controlled by the seed in :class:`TwoCirclesParams`.


In [ ]:
from __future__ import annotations

from dataclasses import dataclass
from typing import Tuple

import numpy as np

from homolipop.delaunay import delaunay_triangulation
from homolipop.pipeline import persistent_homology_from_points
from homolipop.persistence import field_Fp
from homolipop.persistence_rings import RingStyle, plot_rings_from_barcode, red_palette


@dataclass(frozen=True)
class TwoCirclesParams:
    """
    Parameters for sampling two noisy circles in :math:`\\mathbb R^2`.

    Attributes
    ----------
    n_per_circle:
        Number of sample points on each circle.
    radius:
        Circle radius.
    center_left:
        Center of the left circle.
    center_right:
        Center of the right circle.
    noise_sigma:
        Standard deviation :math:`\\sigma` of i.i.d. Gaussian noise added to each coordinate.
        If zero, points lie exactly on the two circles.
    seed:
        Seed for NumPy's random number generator.
    """

    n_per_circle: int = 180
    radius: float = 1.0
    center_left: Tuple[float, float] = (-1.6, 0.0)
    center_right: Tuple[float, float] = (1.6, 0.0)
    noise_sigma: float = 0.03
    seed: int = 0


def sample_two_circles(p: TwoCirclesParams) -> np.ndarray:
    """
    Sample a noisy two circles point cloud.

    Parameters
    ----------
    p:
        Sampling parameters.

    Returns
    -------
    numpy.ndarray
        Array of shape ``(2 * p.n_per_circle, 2)`` whose rows are points in :math:`\\mathbb R^2`.

    Notes
    -----
    The angles on each circle are sampled independently and uniformly from :math:`[0,2\\pi)`.
    If ``p.noise_sigma > 0``, i.i.d. Gaussian noise :math:`\\mathcal N(0,\\sigma^2)` is added
    to each coordinate.
    """
    rng = np.random.default_rng(p.seed)

    theta1 = rng.uniform(0.0, 2.0 * np.pi, size=p.n_per_circle)
    theta2 = rng.uniform(0.0, 2.0 * np.pi, size=p.n_per_circle)

    c1 = np.asarray(p.center_left, dtype=float)
    c2 = np.asarray(p.center_right, dtype=float)

    x1 = c1 + p.radius * np.column_stack((np.cos(theta1), np.sin(theta1)))
    x2 = c2 + p.radius * np.column_stack((np.cos(theta2), np.sin(theta2)))

    x = np.vstack((x1, x2))
    if p.noise_sigma > 0.0:
        x = x + rng.normal(0.0, p.noise_sigma, size=x.shape)

    return x


def main() -> None:
    """
    Compute alpha filtration persistent homology for two circles and save a ring plot.

    The output image is written to ``two_circles_rings.png`` in the current working directory.
    """
    points = sample_two_circles(TwoCirclesParams())

    del_res = delaunay_triangulation(points)
    maximal_simplices = del_res.delaunay_simplices

    res = persistent_homology_from_points(
        points,
        maximal_simplices,
        max_dim=2,
        field=field_Fp(2),
    )

    style = RingStyle(
        cmap=red_palette(),
        gap_fraction=0.02,
        min_width_deg=1.0,
        gamma=0.65,
        edgecolor="#2b0000",
        linewidth=0.6,
        alpha=0.95,
        infinite_scale=1.07,
    )

    fig = plot_rings_from_barcode(
        res.barcode,
        ncols=3,
        style=style,
        suptitle="Two circles, alpha filtration, F_2",
    )
    fig.savefig("two_circles_rings.png", dpi=220, bbox_inches="tight")


if __name__ == "__main__":
    main()